In [11]:
import numpy as np
import itertools
import pandas as pd

In [12]:
# Rooms in the building
rooms = ["Living Room", "Bedroom", "Kitchen", "Office", "Bathroom", "Storage"]

# Search cost for each room
search_cost = {
    "Living Room": 3,
    "Bedroom": 4,
    "Kitchen": 2,
    "Office": 3,
    "Bathroom": 2,
    "Storage": 1
}

# Initial belief over rooms
initial_belief = np.array([0.30, 0.25, 0.15, 0.15, 0.10, 0.05])

gamma = 1.0

In [13]:
step = 0.1
values = np.arange(0, 1 + step, step)

belief_space = []
for probs in itertools.product(values, repeat=len(rooms)):
    if abs(sum(probs) - 1.0) < 1e-6:
        belief_space.append(np.array(probs))

belief_space = np.array(belief_space)

print("Total belief states:", len(belief_space))


Total belief states: 3003


In [14]:
def search_transition(belief, room_index):
    """Returns probability of success and updated belief if not found"""
    success_prob = belief[room_index]

    if success_prob < 1:
        updated = belief.copy()
        updated[room_index] = 0
        updated = updated / (1 - success_prob)
    else:
        updated = belief.copy()

    return success_prob, updated


def search_reward(room_index):
    return -search_cost[rooms[room_index]]


In [15]:
def closest_belief_index(belief):
    distances = np.linalg.norm(belief_space - belief, axis=1)
    return np.argmin(distances)


In [16]:
def evaluate_policy(policy, max_iter=50):
    V = np.zeros(len(belief_space))

    for _ in range(max_iter):
        V_new = np.zeros_like(V)

        for i, belief in enumerate(belief_space):
            action = policy[i]
            p_found, next_belief = search_transition(belief, action)

            if p_found > 0:
                V_new[i] = search_reward(action)
            else:
                next_i = closest_belief_index(next_belief)
                V_new[i] = search_reward(action) + gamma * V[next_i]

        V = V_new

    return V


In [17]:
def improve_policy(V):
    new_policy = np.zeros(len(belief_space), dtype=int)

    for i, belief in enumerate(belief_space):
        action_values = []

        for a in range(len(rooms)):
            p_found, next_belief = search_transition(belief, a)
            if p_found > 0:
                value = search_reward(a)
            else:
                next_i = closest_belief_index(next_belief)
                value = search_reward(a) + gamma * V[next_i]
            action_values.append(value)

        new_policy[i] = np.argmax(action_values)

    return new_policy


In [18]:
policy = np.random.randint(0, len(rooms), size=len(belief_space))
history = []

for iteration in range(30):
    V = evaluate_policy(policy)
    new_policy = improve_policy(V)

    changes = np.sum(policy != new_policy)
    history.append({"iteration": iteration + 1, "policy_changes": int(changes)})

    if changes == 0:
        print("Policy converged in", iteration + 1, "iterations")
        break

    policy = new_policy


Policy converged in 2 iterations


In [19]:
init_idx = closest_belief_index(initial_belief)

print("\n=== FINAL RESULTS ===")
print("Optimal first room to search:", rooms[policy[init_idx]])
print("Expected total search cost:", -V[init_idx])

pd.DataFrame(history)



=== FINAL RESULTS ===
Optimal first room to search: Storage
Expected total search cost: 1.0


,iteration,policy_changes
0,1,2525
1,2,0


In [20]:
print("\nOptimal search sequence:")
belief = initial_belief.copy()

for step in range(len(rooms)):
    idx = closest_belief_index(belief)
    action = policy[idx]
    print(f"Step {step+1}: Search {rooms[action]}")
    _, belief = search_transition(belief, action)



Optimal search sequence:
Step 1: Search Storage
Step 2: Search Kitchen
Step 3: Search Bathroom
Step 4: Search Living Room
Step 5: Search Office
Step 6: Search Bedroom
